In [529]:
import pandas as pd
from fuzzywuzzy import process
import datetime
pd.set_option('display.max_rows', None)

In [530]:
df = pd.read_csv("solicitudes_credito.csv", sep=";", encoding="utf-8")

In [531]:
df.head()

,Unnamed: 0,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,0,masculino,COMERCIO,charcuteria,manrique central no. 2,1,3.0,28/12/2017,7000000,microempresarial
1,1,masculino,comercio,distribuidora de,santa lucia,2,12.0,16/03/2018,"$ 7,000,000.00",microempresarial
2,2,masculino,comercio,tienda,belen,3,16.0,2018/02/14,7800000,microempresarial
3,3,masculino,servicio,organización y,robledo,2,7.0,8/09/2016,1500000,microempresarial
4,4,masculino,servicio,COMIDAS RAPIDAS,20 de julio,2,13.0,20/12/2016,3000000,microempresarial


In [532]:
df.shape

(10920, 10)

In [533]:
df_copia = df.drop_duplicates()
df_copia = df_copia.dropna()
df_copia = df_copia.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_copia.shape

(10716, 10)

In [534]:
df_copia['sexo'] = df_copia['sexo'].str.lower().str.strip()
valores_unicosex = df_copia['sexo'].unique()
valores_unicosex.sort()
len(valores_unicosex)

2

In [535]:
6617 + 3589

10206

In [536]:
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.rstrip() 
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.replace('-', '') 
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.replace('_', ' ') 
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.replace('  ', ' ') 
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.replace(' ', '')  
df_copia['idea_negocio'] = df_copia['idea_negocio'].str.lower() 

In [537]:
valores_unicosin = df_copia['idea_negocio'].unique()
valores_unicosin.sort()
len(valores_unicosin)

75

In [538]:
df_copia['tipo_de_emprendimiento'] = df_copia['tipo_de_emprendimiento'].str.lower().str.strip()
valores_unicosemp = df_copia['tipo_de_emprendimiento'].unique()
valores_unicosemp.sort()
len(valores_unicosemp)

4

In [539]:
debe_dar = 225

In [540]:
df_copia['barrio'] = df_copia['barrio'].str.rstrip()  # Eliminar espacios en blanco al principio y al final
df_copia['barrio'] = df_copia['barrio'].str.replace(r'[-_]', ' ', regex=True)  # Reemplazar guiones y guiones bajos con espacios
df_copia['barrio'] = df_copia['barrio'].str.replace('  ', ' ', regex=True)
df_copia['barrio'] = df_copia['barrio'].str.replace('santo domingo savio ', 'santo domingo savio', regex=True)
df_copia['barrio'] = df_copia['barrio'].str.replace('cabecera san antonio ', 'cabecera san antonio', regex=True)
df_copia['barrio'] = df_copia['barrio'].str.replace('playon de los ', 'playon de los', regex=True)
df_copia['barrio'] = df_copia['barrio'].str.rstrip() 
print(len(df_copia['barrio'].unique()))  # Debería imprimir 225
valores_unicosbarr = df_copia['barrio'].unique()
valores_unicosbarr.sort() 
valores_unicosbarr

223


array(['13 de noviembre', '20 de julio', 'aguas frias',
       'alejandro echavarria', 'alfonso lopez', 'altamira', 'altavista',
       'andalucia', 'antonio nari¿o', 'antonio nariño', 'aranjuez',
       'aures no. 2', 'aures no.1', 'barrio caicedo', 'barrio colombia',
       'barrio colon', 'barrio cristobal', 'barrio de jesus',
       'barro blanco', 'belalcazar', 'belen', 'belencito',
       'bello horizonte', 'bel¿n', 'betania', 'blanquizal',
       'bombona no. 1', 'bombona no. 2', 'bosques de san pablo', 'boston',
       'boyaca', 'brasilia', 'buenos aires', 'cabecera san antonio',
       'cabecera san cristobal', 'calasanz', 'calasanz parte alta',
       'campo alegre', 'campo amor', 'campo valdes no. 1',
       'campo valdes no. 2', 'caribe', 'carlos e. restrepo', 'carpinelo',
       'castilla', 'cataluna', 'centro admtivo', 'cordoba', 'cristo rey',
       'doce de octubre no.1', 'doce de octubre no.2', 'eduardo santos',
       'el astillero', 'el chagualo', 'el compromiso', 'e

In [541]:
print(len(df_copia['estrato'].unique()))
valores_unicosest = df_copia['estrato'].unique()
valores_unicosest.sort()
valores_unicosest

4


array([0, 1, 2, 3], dtype=int64)

In [542]:
print(len(df_copia['comuna_ciudadano'].unique())) 
df_copia['comuna_ciudadano'] = df_copia['comuna_ciudadano'].astype('Int64')
valores_unicom = df_copia['comuna_ciudadano'].unique()
valores_unicom

21


<IntegerArray>
[3, 12, 16, 7, 13, 70, 4, 9, 6, 1, 2, 8, 80, 5, 10, 15, 60, 90, 50, 14, 11]
Length: 21, dtype: Int64

In [543]:
def clean_date(date_str):
    # List of possible date formats
    date_formats = [
        "%Y-%m-%d",  # Year-Month-Day
        "%d-%m-%Y",  # Day-Month-Year
        "%m-%d-%Y",  # Month-Day-Year
        "%Y/%m/%d",  # Year/Month/Day
        "%d/%m/%Y",  # Day/Month/Year
        "%m/%d/%Y",  # Month/Day/Year
    ]

    cleaned_date = None
    for fmt in date_formats:
        try:
            cleaned_date = datetime.datetime.strptime(date_str, fmt).date()
            break  # Stop loop if a valid date is found
        except ValueError:
            continue  # Continue to next format if parsing fails

    return cleaned_date

# Aplica la función clean_date a la columna 'fecha_de_beneficio' del DataFrame
df_copia['fecha_de_beneficio'] = df_copia['fecha_de_beneficio'].apply(clean_date)
print(len(df_copia['fecha_de_beneficio'].unique())) 

795


In [544]:
debe_ser_f = 795 
debe_ser_f

795

In [545]:
def clean_numeric_column(column):
    column = column.astype(str)
    column = column.str.replace('$', '').str.replace(',', '')
    column = pd.to_numeric(column, errors='coerce')
    column = column.dropna()
    return column


df_copia['monto_del_credito'] = clean_numeric_column(df_copia['monto_del_credito'])
valores_unicomon = clean_numeric_column(df_copia['monto_del_credito']).unique()
valores_unicomon
len(valores_unicomon)

C:\Users\javie\AppData\Local\Temp\ipykernel_10036\3974151914.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  column = column.str.replace('$', '').str.replace(',', '')


277

In [546]:
df_copia['línea_credito'] = df_copia['línea_credito'].str.lower()
df_copia['línea_credito'] = df_copia['línea_credito'].str.rstrip()
df_copia['línea_credito'] = df_copia['línea_credito'].str.replace('-', '') 
df_copia['línea_credito'] = df_copia['línea_credito'].str.replace('_', ' ')
df_copia['línea_credito'] = df_copia['línea_credito'].str.replace(' ', '') 
valores_unicolc = df_copia['línea_credito'].unique()
print(valores_unicolc)
len(valores_unicolc)

['microempresarial' 'juridicaycap.semilla' 'fomentoagropecuario'
 'agropecuaria' 'empresarialed.' 'solidaria' 'credioportuno' 'solidiaria'
 'ayacuchoformal']


9

In [547]:
df_copia1 = df_copia.drop(columns=['Unnamed: 0'])
df_copia1 = df_copia1.dropna(how='all')
df_copia1 = df_copia1.drop_duplicates()

In [548]:
df_copia1.shape

(10206, 9)

In [549]:
df_copia1.head()

,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,masculino,comercio,charcuteria,manrique central no. 2,1,3,2017-12-28,7000000.0,microempresarial
1,masculino,comercio,distribuidorade,santa lucia,2,12,2018-03-16,7000000.0,microempresarial
2,masculino,comercio,tienda,belen,3,16,2018-02-14,7800000.0,microempresarial
3,masculino,servicio,organizacióny,robledo,2,7,2016-09-08,1500000.0,microempresarial
4,masculino,servicio,comidasrapidas,20 de julio,2,13,2016-12-20,3000000.0,microempresarial


In [551]:
barrio_counts = df_copia1["línea_credito"].value_counts()

print(barrio_counts)

microempresarial        10020
empresarialed.             70
agropecuaria               55
juridicaycap.semilla       33
credioportuno              21
fomentoagropecuario         4
solidaria                   1
solidiaria                  1
ayacuchoformal              1
Name: línea_credito, dtype: int64
